# Fix Image Color Profile

The input image `test_image2.jpg` has a Cyan color profile (High Blue, High Green, Low Red).
HoVerNet expects H&E (Purple/Pink), which means High Blue, High Red, Low Green.

We will swap the Red and Green channels to transform Cyan -> Purple.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import scipy.io

INPUT_PATH = "test_input/test_image2.jpg"
OUTPUT_DIR = "output_fix_purple"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Load Image
img = cv2.imread(INPUT_PATH) # BGR
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(f"Original Mean: {img.mean(axis=(0,1))}")

# 2. Swap R and G
# img is RGB. R=0, G=1, B=2
img_purple = img.copy()
img_purple[..., 0] = img[..., 1] # New R = Old G (High)
img_purple[..., 1] = img[..., 0] # New G = Old R (Low)
# B stays same

print(f"Purple Mean: {img_purple.mean(axis=(0,1))}")

# Save as BGR for cv2
img_purple_bgr = cv2.cvtColor(img_purple, cv2.COLOR_RGB2BGR)
purple_path = "test_input/test_image2_purple_nb.jpg"
cv2.imwrite(purple_path, img_purple_bgr)

# 3. Run Inference
cmd = [
    "python", "run_custom.py",
    purple_path,
    "--output_dir", OUTPUT_DIR,
    "--model_mode", "fast"
]
# We filter stdout to avoid huge logs but capture stderr
print("Running inference...")
subprocess.check_call(cmd)
print("Inference done.")

# 4. Check Results
mat_path = os.path.join(OUTPUT_DIR, 'mat', 'test_image2_purple_nb.mat')
if os.path.exists(mat_path):
    data = scipy.io.loadmat(mat_path)
    inst_map = data['inst_map']
    num_cells = len(np.unique(inst_map)) - 1
    print(f"Detected {num_cells} cells!")
else:
    print("No output .mat file found.")

Original Mean: [189.54113233 211.98704186 233.59815201]
Purple Mean: [211.98704186 189.54113233 233.59815201]
Running inference...


|2026-02-14|18:19:26.375| [INFO] .... Detect #GPUS: 1


DEBUG: Loaded image /Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/output_fix_purple/_temp_input/test_image2_purple_nb.jpg, shape=(1080, 1920, 3), min=0, max=255, mean=211.6842589377572


Process Patches:   0%|                                   | 0/22 [00:00<?, ?it/s]

Process Patches:   5%|#2                         | 1/22 [00:19<06:43, 19.20s/it]

...

Process Patches:  91%|#######################6  | 20/22 [04:40<00:15,  7.77s/it]

Process Patches:  95%|########################8 | 21/22 [04:47<00:13, 13.69s/it]


DEBUG: NP Logits stats: min=-8.687297821044922, max=4.707661151885986, mean=-1.9518085718154907
DEBUG: NP Logits channel 0 mean=4.560405731201172, channel 1 mean=-8.464022636413574
DEBUG: NP Logits stats: min=-8.687297821044922, max=4.707661151885986, mean=-1.9518085718154907
DEBUG: NP Logits channel 0 mean=4.560405731201172, channel 1 mean=-8.464022636413574
DEBUG: NP Logits stats: min=-8.687297821044922, max=4.707661151885986, mean=-1.9518085718154907
DEBUG: NP Logits channel 0 mean=4.560405731201172, channel 1 mean=-8.464022636413574
DEBUG: Patch 16 mean=239.37579854329428 min=50
DEBUG: NP Logits stats: min=-10.552436828613281, max=7.099593162536621, mean=-1.9355707168579102
DEBUG: NP Logits channel 0 mean=4.568655014038086, channel 1 mean=-8.439796447753906
DEBUG: Patch 17 mean=214.96833292643228 min=35
DEBUG: Patch 18 mean=219.26754252115884 min=42
DEBUG: Patch 19 mean=229.9663340250651 min=38
DEBUG: NP Logits stats: min=-11.509313583374023, max=10.80647087097168, mean=-1.13765573

|2026-02-14|18:24:26.060| [INFO] ........................ Done Assembling test_image2_purple_nb


Using Conda Python: /Users/Hrithik/miniconda3/envs/hovernet_env/bin/python
Running command: /Users/Hrithik/miniconda3/envs/hovernet_env/bin/python run_infer.py --model_path=/Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/checkpoints/hovernet_fast_pannuke_type_tf2pytorch.tar --model_mode=fast --nr_types=6 --batch_size=4 --nr_inference_workers=0 --type_info_path=/Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/type_info.json tile --save_raw_map --input_dir=/Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/output_fix_purple/_temp_input --output_dir=/Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/output_fix_purple

Inference complete. Results in /Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/output_fix_purple
Overlay image: /Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/output_fix_purple/overlay/test_image2_purple_nb.jpg.png
JSON data: /Users/Hrithik/.gemini/antigravity/scratch/hover_net_inference/outp